In [ ]:
from datasets import load_dataset,load_from_disk
from sentence_transformers import SentenceTransformer, util
from pyarrow import json
from pathlib import Path
import pyarrow as pa
from datasets import load_dataset,load_from_disk
import numpy as np

In [ ]:
experiments_root = Path('../resources/preprocessed_data/wikilarge_original')

In [ ]:
data = load_from_disk(experiments_root)

In [ ]:
data

In [ ]:
#data = data.select(range(1000))

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
def calculate_similarity(row):
    embeddings1 = model.encode(row['original_text'], convert_to_tensor=True)
    embeddings2 = model.encode(row['simple_text'], convert_to_tensor=True)
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    row['similarity'] = cosine_scores[0][0].detach().cpu().item()
    print(row)
    return row

In [ ]:
data = data.map(calculate_similarity)

In [ ]:
data.save_to_disk('../resources/preprocessed_data/wikilarge_sim')

In [ ]:
data

Experiments with different values of similarity

In [ ]:
data = load_from_disk('../resources/preprocessed_data/wikilarge_sim')

In [ ]:
data

In [ ]:
data_df = data.to_pandas()

In [ ]:
data_df

In [ ]:
data_test = data_df[(data_df['similarity'] > 0.5)]

In [ ]:
data_test

In [ ]:
from datasets import Dataset

In [ ]:
data_pandas = Dataset.from_pandas(data_test)

In [ ]:
data_pandas

In [ ]:
data_pandas = data_pandas.remove_columns('__index_level_0__')

In [ ]:
data_pandas

In [ ]:
data_pandas.save_to_disk('../resources/preprocessed_data/wikilarge_sim0.7')